In [158]:
project = 'BitcoinWallet'
# 'JHotDraw', 'JEdit', 'SweetHome3D', 'BitcoinWallet', 'K9Mail'
prjfile = project.lower()

In [159]:
import pandas as pd

nodes = pd.read_csv(f"{prjfile}-cd-nodes.csv")
# edges = pd.read_csv(f"{prjfile}-cd-edges.csv")

nodes.shape #, edges.shape

(15, 14)

In [160]:
nodes['level'] = [l if l==0 else 1 for l in nodes['level']]

note: 

communities [louvain] uses the parameters:

- algorithm = Louvain
- weight column = weight
- parameters:
  - directed = True
  - overlapping = False
  - random gen seed = 42
  - resolution = 1 for projects with > 200 classes, 2 otherwise
  - deep hierarchy = False

In [161]:
nodes['members'] = [set(l.split(" ")) for l in nodes['CD_MemberList']]

nodes.sort_values(by=['level'], ascending=False, inplace=True)
indices = list(nodes.index)

df = nodes[['name', 'CD_MemberList_Size', 'members', 'level']]
pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,level
1,C120431,36,"{BlockchainService, BlockchainService.LocalBin...",1.0
2,C120432,28,"{TransactionsAdapter.ChangeType, ExchangeRates...",1.0
3,C120433,28,"{Io, Crypto, MaintenanceDialogFragment.State, ...",1.0
4,C120434,23,"{InputParser, RestoreWalletFromExternalDialogF...",1.0
5,C120435,15,"{ExchangeRatesProvider, ExchangeRate, Settings...",1.0
11,C120441,6,"{RequestCoinsFragment, CurrencyAmountView, Cur...",1.0
12,C120442,4,"{RequestWalletBalanceTask, RequestWalletBalanc...",1.0
13,C120443,4,"{DirectPaymentTask.BluetoothPaymentTask, Direc...",1.0
14,C120444,2,"{RequestWalletBalanceTask.ElectrumServer.Type,...",1.0
0,C120445,196,"{BlockListAdapter.ListItem.ListTransaction, Se...",0.0


note: level currently assumes non-deep communities with manual editing in cytoscape. should be automated.

problem: cytoscape won't export community edges??

- visited nodes <- []
- remaining nodes <- all nodes
- find sink (node with no outgoing edges/node with most members)
- assign level 0 to sink
- add sink to visited nodes
- remove sink from remaining nodes
- while there are remaining nodes:
  - increment level
  - for nodes in remaining nodes that has edge to visited nodes
    - assign level to those nodes
    - add those nodes to visited nodes
    - remove those nodes from remaining nodes

In [162]:
nodes['my_members_str'] = ["" for i in indices]
df = nodes[['name', 'CD_MemberList_Size', 'members', 'my_members_str']]
pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,my_members_str
1,C120431,36,"{BlockchainService, BlockchainService.LocalBin...",
2,C120432,28,"{TransactionsAdapter.ChangeType, ExchangeRates...",
3,C120433,28,"{Io, Crypto, MaintenanceDialogFragment.State, ...",
4,C120434,23,"{InputParser, RestoreWalletFromExternalDialogF...",
5,C120435,15,"{ExchangeRatesProvider, ExchangeRate, Settings...",
11,C120441,6,"{RequestCoinsFragment, CurrencyAmountView, Cur...",
12,C120442,4,"{RequestWalletBalanceTask, RequestWalletBalanc...",
13,C120443,4,"{DirectPaymentTask.BluetoothPaymentTask, Direc...",
14,C120444,2,"{RequestWalletBalanceTask.ElectrumServer.Type,...",
0,C120445,196,"{BlockListAdapter.ListItem.ListTransaction, Se...",


In [163]:
visited = { n for n in nodes.loc[indices[0],'members'] }
nodes.loc[indices[0],'my_members_str'] = ' '.join(visited)


In [164]:
for idx in range(1,len(indices)):
  i = indices[idx-1]
  j = indices[idx]
  my_nodes = nodes.loc[j,'members'] - visited
  visited |= my_nodes
  # print(nodes.loc[j,'name'])
  # print("then",nodes.loc[j,'members'])
  # print(" now",my_nodes)
  nodes.loc[j,'my_members_str'] = ' '.join(my_nodes)

nodes['my_members'] = [set(l.split(" ")) if l else set() for l in nodes['my_members_str']]
nodes['my_members_size'] = [len(l) for l in nodes['my_members']]


In [165]:
df = nodes[['name', 'CD_MemberList_Size', 'members', 'my_members_size', 'my_members', 'level']]

pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,my_members_size,my_members,level
1,C120431,36,"{BlockchainService, BlockchainService.LocalBin...",36,"{BlockchainService, BlockchainService.LocalBin...",1.0
2,C120432,28,"{TransactionsAdapter.ChangeType, ExchangeRates...",28,"{TransactionsAdapter.ChangeType, ExchangeRates...",1.0
3,C120433,28,"{Io, Crypto, MaintenanceDialogFragment.State, ...",28,"{Io, Crypto, MaintenanceDialogFragment.State, ...",1.0
4,C120434,23,"{InputParser, RestoreWalletFromExternalDialogF...",23,"{InputParser, RestoreWalletFromExternalDialogF...",1.0
5,C120435,15,"{ExchangeRatesProvider, ExchangeRate, Settings...",15,"{ExchangeRatesProvider, ExchangeRate, Settings...",1.0
11,C120441,6,"{RequestCoinsFragment, CurrencyAmountView, Cur...",6,"{RequestCoinsFragment, CurrencyAmountView, Cur...",1.0
12,C120442,4,"{RequestWalletBalanceTask, RequestWalletBalanc...",4,"{RequestWalletBalanceTask, RequestWalletBalanc...",1.0
13,C120443,4,"{DirectPaymentTask.BluetoothPaymentTask, Direc...",4,"{DirectPaymentTask.BluetoothPaymentTask, Direc...",1.0
14,C120444,2,"{RequestWalletBalanceTask.ElectrumServer.Type,...",2,"{RequestWalletBalanceTask.ElectrumServer.Type,...",1.0
0,C120445,196,"{BlockListAdapter.ListItem.ListTransaction, Se...",0,{},0.0


In [166]:
from functools import reduce

all = reduce(lambda x,y: x|y, nodes['my_members'])
len(all)

196

In [167]:
nodes.columns

Index(['CD_AnnotatedMembers', 'CD_AnnotatedMembers_Overlap',
       'CD_AnnotatedMembers_Pvalue', 'CD_AnnotatedMembers_Size',
       'CD_CommunityName', 'CD_Labeled', 'CD_MemberList',
       'CD_MemberList_LogSize', 'CD_MemberList_Size', 'level', 'name',
       'selected', 'shared name', 'wordcloud.sync', 'members',
       'my_members_str', 'my_members', 'my_members_size'],
      dtype='object')

In [168]:
c_nodes = nodes.drop(nodes[nodes.my_members_str.map(len)==0].index)
c_nodes = c_nodes.drop(columns=[
      'CD_AnnotatedMembers', 'CD_AnnotatedMembers_Overlap',
      'CD_AnnotatedMembers_Pvalue', 'CD_AnnotatedMembers_Size',
      'CD_CommunityName', 'CD_Labeled', 'CD_MemberList',
      'CD_MemberList_LogSize', 'CD_MemberList_Size',
      'selected', 'shared name', 'members', 'my_members_str', 
      'my_members_size'])
c_nodes.rename(columns={'name':'id', 'my_members':'members'}, inplace=True)
df = c_nodes
pd.concat([df.head(5), df.tail(5)])

,level,id,wordcloud.sync,members
1,1.0,C120431,NaN,"{BlockchainService, BlockchainService.LocalBin..."
2,1.0,C120432,NaN,"{TransactionsAdapter.ChangeType, ExchangeRates..."
3,1.0,C120433,NaN,"{Io, Crypto, MaintenanceDialogFragment.State, ..."
4,1.0,C120434,NaN,"{InputParser, RestoreWalletFromExternalDialogF..."
5,1.0,C120435,NaN,"{ExchangeRatesProvider, ExchangeRate, Settings..."
10,1.0,C120440,NaN,"{ScanActivity, CameraManager, ScanActivity.Aut..."
11,1.0,C120441,NaN,"{RequestCoinsFragment, CurrencyAmountView, Cur..."
12,1.0,C120442,NaN,"{RequestWalletBalanceTask, RequestWalletBalanc..."
13,1.0,C120443,NaN,"{DirectPaymentTask.BluetoothPaymentTask, Direc..."
14,1.0,C120444,NaN,"{RequestWalletBalanceTask.ElectrumServer.Type,..."


note: community nodes are named using 3 most recurring words in member class names

other ideas:

- use the name of the largest class
- use the name of "central" class in the community (e.g., most ingoing + outgoing edges?)

In [169]:
import re
from collections import Counter

names = ['-'.join([name for name, _ in Counter(re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', ' '.join(members))).most_common(3)]) for members in c_nodes['members']]

words = [' '.join(re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', ' '.join(members))) for members in c_nodes['members']]

In [170]:
c_nodes['name'] = names
c_nodes['words'] = words
c_nodes['id'] = names
c_nodes['members_str'] = [' '.join(m) for m in c_nodes['members']]
# for line in [' '.join(s) for s in c_nodes['members']]:
#   print(line)
df = c_nodes

pd.concat([df.head(5), df.tail(5)])

,level,id,wordcloud.sync,members,name,words,members_str
1,1.0,Live-Data-Wallet,NaN,"{BlockchainService, BlockchainService.LocalBin...",Live-Data-Wallet,Blockchain Service Blockchain Service Local Bi...,BlockchainService BlockchainService.LocalBinde...
2,1.0,View-Wallet-Fragment,NaN,"{TransactionsAdapter.ChangeType, ExchangeRates...",View-Wallet-Fragment,Transactions Adapter Change Type Exchange Rate...,TransactionsAdapter.ChangeType ExchangeRatesFr...
3,1.0,Fragment-Wallet-Dialog,NaN,"{Io, Crypto, MaintenanceDialogFragment.State, ...",Fragment-Wallet-Dialog,Io Crypto Maintenance Dialog Fragment State Bl...,Io Crypto MaintenanceDialogFragment.State Bloc...
4,1.0,Wallet-Input-Parser,NaN,"{InputParser, RestoreWalletFromExternalDialogF...",Wallet-Input-Parser,Input Parser Restore Wallet From External Dial...,InputParser RestoreWalletFromExternalDialogFra...
5,1.0,Wallet-Address-Live,NaN,"{ExchangeRatesProvider, ExchangeRate, Settings...",Wallet-Address-Live,Exchange Rates Provider Exchange Rate Settings...,ExchangeRatesProvider ExchangeRate SettingsFra...
10,1.0,Scan-Activity-View,NaN,"{ScanActivity, CameraManager, ScanActivity.Aut...",Scan-Activity-View,Scan Activity Camera Manager Scan Activity Aut...,ScanActivity CameraManager ScanActivity.AutoFo...
11,1.0,Currency-View-Amount,NaN,"{RequestCoinsFragment, CurrencyAmountView, Cur...",Currency-View-Amount,Request Coins Fragment Currency Amount View Cu...,RequestCoinsFragment CurrencyAmountView Curren...
12,1.0,Request-Wallet-Balance,NaN,"{RequestWalletBalanceTask, RequestWalletBalanc...",Request-Wallet-Balance,Request Wallet Balance Task Request Wallet Bal...,RequestWalletBalanceTask RequestWalletBalanceT...
13,1.0,Payment-Task-Direct,NaN,"{DirectPaymentTask.BluetoothPaymentTask, Direc...",Payment-Task-Direct,Direct Payment Task Bluetooth Payment Task Dir...,DirectPaymentTask.BluetoothPaymentTask DirectP...
14,1.0,Request-Wallet-Balance,NaN,"{RequestWalletBalanceTask.ElectrumServer.Type,...",Request-Wallet-Balance,Request Wallet Balance Task Electrum Server Ty...,RequestWalletBalanceTask.ElectrumServer.Type R...


In [171]:

o_nodes = pd.read_csv(f"{prjfile}-nodes.csv")
o_edges = pd.read_csv(f"{prjfile}-edges.csv")

remains = o_nodes[~o_nodes['name'].isin(all)].copy()
remains.drop(columns=['type'], inplace=True)
remains['level'] = [0 for _ in remains['name']]
remains['id'] = remains['name']
remains['members'] = [set([name]) for name in remains['name']]
remains['members_str'] = remains['name']
words = [' '.join(re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', ' '.join(members))) for members in remains['members']]
remains['words'] = words

remains

,id,name,package,level,members,members_str,words
1,Constants,Constants,de.schildbach.wallet,0,{Constants},Constants,Constants
2,Constants.Files,Constants.Files,de.schildbach.wallet,0,{Constants.Files},Constants.Files,Constants Files
3,Logging,Logging,de.schildbach.wallet,0,{Logging},Logging,Logging
50,AddressBookActivity.TwoFragmentAdapter,AddressBookActivity.TwoFragmentAdapter,de.schildbach.wallet.ui,0,{AddressBookActivity.TwoFragmentAdapter},AddressBookActivity.TwoFragmentAdapter,Address Book Activity Two Fragment Adapter
52,BitmapFragment,BitmapFragment,de.schildbach.wallet.ui,0,{BitmapFragment},BitmapFragment,Bitmap Fragment
61,DividerItemDecoration,DividerItemDecoration,de.schildbach.wallet.ui,0,{DividerItemDecoration},DividerItemDecoration,Divider Item Decoration
74,HelpDialogFragment,HelpDialogFragment,de.schildbach.wallet.ui,0,{HelpDialogFragment},HelpDialogFragment,Help Dialog Fragment
82,ProgressDialogFragment,ProgressDialogFragment,de.schildbach.wallet.ui,0,{ProgressDialogFragment},ProgressDialogFragment,Progress Dialog Fragment
95,ShowPasswordCheckListener,ShowPasswordCheckListener,de.schildbach.wallet.ui,0,{ShowPasswordCheckListener},ShowPasswordCheckListener,Show Password Check Listener
96,StickToTopLinearLayoutManager,StickToTopLinearLayoutManager,de.schildbach.wallet.ui,0,{StickToTopLinearLayoutManager},StickToTopLinearLayoutManager,Stick To Top Linear Layout Manager


In [172]:
c_nodes = c_nodes.append(remains, ignore_index = True)
df = c_nodes

pd.concat([df.head(5), df.tail(5)])

,level,id,wordcloud.sync,members,name,words,members_str,package
0,1.0,Live-Data-Wallet,NaN,"{BlockchainService, BlockchainService.LocalBin...",Live-Data-Wallet,Blockchain Service Blockchain Service Local Bi...,BlockchainService BlockchainService.LocalBinde...,NaN
1,1.0,View-Wallet-Fragment,NaN,"{TransactionsAdapter.ChangeType, ExchangeRates...",View-Wallet-Fragment,Transactions Adapter Change Type Exchange Rate...,TransactionsAdapter.ChangeType ExchangeRatesFr...,NaN
2,1.0,Fragment-Wallet-Dialog,NaN,"{Io, Crypto, MaintenanceDialogFragment.State, ...",Fragment-Wallet-Dialog,Io Crypto Maintenance Dialog Fragment State Bl...,Io Crypto MaintenanceDialogFragment.State Bloc...,NaN
3,1.0,Wallet-Input-Parser,NaN,"{InputParser, RestoreWalletFromExternalDialogF...",Wallet-Input-Parser,Input Parser Restore Wallet From External Dial...,InputParser RestoreWalletFromExternalDialogFra...,NaN
4,1.0,Wallet-Address-Live,NaN,"{ExchangeRatesProvider, ExchangeRate, Settings...",Wallet-Address-Live,Exchange Rates Provider Exchange Rate Settings...,ExchangeRatesProvider ExchangeRate SettingsFra...,NaN
42,0.0,Formats,NaN,{Formats},Formats,Formats,Formats,de.schildbach.wallet.util
43,0.0,MonetarySpannable,NaN,{MonetarySpannable},MonetarySpannable,Monetary Spannable,MonetarySpannable,de.schildbach.wallet.util
44,0.0,Nfc,NaN,{Nfc},Nfc,Nfc,Nfc,de.schildbach.wallet.util
45,0.0,Toast,NaN,{Toast},Toast,Toast,Toast,de.schildbach.wallet.util
46,0.0,WholeStringBuilder,NaN,{WholeStringBuilder},WholeStringBuilder,Whole String Builder,WholeStringBuilder,de.schildbach.wallet.util


In [173]:
connections = list()

for _, x in c_nodes.iterrows():
  for _, y in c_nodes.iterrows():
    x_nodes = list(o_nodes.loc[o_nodes['name'].isin(x['members'])]['id'])
    y_nodes = list(o_nodes.loc[o_nodes['name'].isin(y['members'])]['id'])
    x2y = list(o_edges.loc[o_edges['source'].isin(x_nodes) & o_edges['target'].isin(y_nodes)]['interaction'])
    for interaction in x2y:
      connections.append(f"{x['name']} {y['name']} {interaction}")
      # {
      #   'source': x['name'],
      #   'target': y['name'],
      #   'interaction': interaction,
      # }

connections = Counter(connections)
connections = [f"{k} {v}" for k,v in connections.items()]


In [174]:

connections = [tuple(row.split(' ')) for row in connections]
connections = [{'source':s, 'target':t, 'interaction':i, 'weight':c} for s,t,i,c in connections if s!=t]

c_edges = pd.DataFrame(connections)

df = c_edges
pd.concat([df.head(5), df.tail(5)])

,source,target,interaction,weight
0,Live-Data-Wallet,View-Wallet-Fragment,returns,1
1,Live-Data-Wallet,View-Wallet-Fragment,constructs,2
2,Live-Data-Wallet,Fragment-Wallet-Dialog,holds,3
3,Live-Data-Wallet,Fragment-Wallet-Dialog,depends,3
4,Live-Data-Wallet,Wallet-Input-Parser,depends,2
98,Currency-View-Amount,Fragment-Wallet-Dialog,holds,1
99,Currency-View-Amount,Wallet-Address-Live,depends,1
100,Currency-View-Amount,Wallet-Address-Live,holds,1
101,Currency-View-Amount,Request-Payment-Task,depends,2
102,Payment-Task-Direct,Request-Payment-Task,depends,1


In [175]:
o_nodes['community'] = [next(c['id'] for c in c_nodes.to_dict('records') if n in c['members']) for n in o_nodes['name']]

o_nodes

,id,name,package,type,community
0,de.schildbach.wallet.Configuration,Configuration,de.schildbach.wallet,Structure,Wallet-Address-Live
1,de.schildbach.wallet.Constants,Constants,de.schildbach.wallet,Structure,Constants
2,de.schildbach.wallet.Constants.Files,Constants.Files,de.schildbach.wallet,Structure,Constants.Files
3,de.schildbach.wallet.Logging,Logging,de.schildbach.wallet,Structure,Logging
4,de.schildbach.wallet.WalletApplication,WalletApplication,de.schildbach.wallet,Structure,Live-Data-Wallet
...,...,...,...,...,...
224,de.schildbach.wallet.util.Toast,Toast,de.schildbach.wallet.util,Structure,Toast
225,de.schildbach.wallet.util.ViewPagerTabs,ViewPagerTabs,de.schildbach.wallet.util,Structure,Fragment-Wallet-Dialog
226,de.schildbach.wallet.util.WalletUtils,WalletUtils,de.schildbach.wallet.util,Structure,Wallet-Input-Parser
227,de.schildbach.wallet.util.WalletUtils.Monospac...,WalletUtils.MonospaceSpan,de.schildbach.wallet.util,Structure,Fragment-Wallet-Dialog


In [176]:
o_nodes.to_csv(f"{prjfile}-nodes-withcommunities.csv")

In [177]:
c_nodes[['id', 'name', 'words', 'members_str']].to_csv(f"{prjfile}-communities-nodes.csv")
c_edges.to_csv(f"{prjfile}-communities-edges.csv")

# Export to Cytoscape

In [ ]:
import requests

exec(open("p4c_init.py").read())
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

In [ ]:
import py4cytoscape as p4c

p4c.cytoscape_ping()
p4c.cytoscape_version_info()

In [ ]:
p4c.create_network_from_data_frames(c_nodes.drop(columns=['members']), c_edges, title=f"{project}-abstract", collection=project)

In [ ]:
p4c.set_visual_style('SAR')
p4c.notebook_export_show_image()